***Assignment 3***

**Written By: Luke Perry and Devon Warner**

import matplotlib.pyplot as plt
import numpy as np
from pyedflib import highlevel
from scipy.signal import spectrogram, welch, find_peaks
import pandas as pd


In [251]:
import math
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyedflib import highlevel
from scipy.signal import spectrogram, welch

from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [267]:
def get_annot(header, signal):
    """
    Function to translate .edf annotation files into a vector with classes related to sleep state
    
    0 - Awake (W)
    1 - Sleep 1 (1)
    2 - Sleep 2 (2)
    3 - Sleep 3 (3)
    4 - Sleep 4 (4)
    5 - REM (R)
    nan - Not annotated (?)
    """
    
    list_of_annot = header['annotations']

    
    annot = np.zeros((len(signal),1))


    for ann in list_of_annot:

        if 'W' in ann[2]:
            cat = 0
        elif '1' in ann[2]:
            cat = 1
        elif '2' in ann[2]:
            cat = 2
        elif  '3' in ann[2]:
            cat = 3
        elif '4' in ann[2]:
            cat = 4
        elif 'R' in ann[2]:
            cat = 5
        elif '?' in ann[2]:
            cat = 6
        else:
            cat = 6
            
        try:
            annot[int(ann[0]):int(ann[0]) + int(ann[1])] = cat
        except: 
            pass
        
        
        
    return annot

In [268]:
def band_power(data):
    Fs = 100
    freq, time, Sxx= spectrogram(data, Fs, nperseg=100, noverlap=0, scaling='spectrum')
    Sxx = Sxx.T

    delta_band = np.zeros_like(time)
    theta_band = np.zeros_like(time) 
    alpha_band = np.zeros_like(time) 
    beta_band = np.zeros_like(time)
    # print(f'Expected length of time = {(T2-T1)/100}')
    # print(f'len(time) = {len(time)}')
    for i,t in enumerate(time):
        current_Sxx = Sxx[i]
        for f in freq:
            if f<=4:
                delta_band[i]+=current_Sxx[freq==f]
            elif f<8:
                theta_band[i]+=current_Sxx[freq==f]
            elif f<=13:
                alpha_band[i]+=current_Sxx[freq==f]
            elif f<=50:
                beta_band[i]+=current_Sxx[freq==f]
                
    return delta_band, theta_band, alpha_band, beta_band

# Mean of the absolute value of the data
def avg(data):
    return np.mean(np.abs(data))

# Range of the absolute of the magnitude
def rang(data):
    print(data)
    return max(abs(np.double(data)))-min(abs(np.double(data)))

# Variance
def var(data):
    return np.var(data)

In [269]:
def get_bands_from_file(PSG_Filename, Hyp_Filename, label):
    #read the data 
    signals, signal_headers, header = highlevel.read_edf(PSG_Filename)
    #read the annotations 
    signals_hyp, signal_headers_hyp, header_hyp = highlevel.read_edf(Hyp_Filename)
    #use function to translate .edf annotation file into a vector with annotations 
    annot = get_annot(header_hyp, signals[0])
    
    data = signals[0]
    delta_band, theta_band, alpha_band, beta_band = band_power(data)
    
    delta_band = pd.DataFrame(delta_band, columns=['Delta band'])
    theta_band = pd.DataFrame(theta_band, columns=['Theta band'])
    alpha_band = pd.DataFrame(alpha_band, columns=['Alpha band'])
    beta_band = pd.DataFrame(beta_band, columns=['Beta band'])

    T = len(data)
    t = int(T/100)
    annot = annot[0:t]
    annot = pd.DataFrame(annot, columns=['class'])
    
    features = pd.concat([annot, delta_band, theta_band, alpha_band, beta_band], axis=1)
    return features

In [270]:
features1 = get_bands_from_file('SC4001E0-PSG.edf', 'SC4001EC-Hypnogram.edf', 1)
features2 = get_bands_from_file('SC4002E0-PSG.edf', 'SC4002EC-Hypnogram.edf', 1)
features = pd.concat([features1, features2], axis=0)
print(features)

       class   Delta band  Theta band  Alpha band   Beta band
0        0.0    66.977903    2.023953    2.829123    4.795149
1        0.0   719.507739   80.449050    8.819837   15.816797
2        0.0   819.491009   63.250742   32.428322   51.670797
3        0.0   306.345809  526.242716   63.723784   57.499063
4        0.0  1902.389226  369.237354   43.094873   20.900476
...      ...          ...         ...         ...         ...
84895    0.0   594.521916   73.190658   36.536285  140.362038
84896    0.0   326.105746   17.665097   26.668790   98.177925
84897    0.0   690.802196   39.557094   19.180396  185.077684
84898    0.0   220.117383   93.217599   35.718400   65.732486
84899    0.0   369.682504   77.622151   37.416572  133.985574

[164400 rows x 5 columns]


In [271]:
features1 = get_bands_from_file('SC4001E0-PSG.edf', 'SC4001EC-Hypnogram.edf', 1)
features2 = get_bands_from_file('SC4002E0-PSG.edf', 'SC4002EC-Hypnogram.edf', 1)
X = pd.concat([features1, features2], axis=0)
y = np.ravel(X['class'])                       # Annotations: sleep or wake
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size = 0.40, random_state = 1)

print('Training set dimensions')
print(trainX.shape)
print(trainY.shape)

print('\nTest set dimensions')
print(testX.shape)
print(testY.shape)


scaler = StandardScaler()           # To normalize the data
# model = GaussianNB()                        # classifier: naive Bayes
# model = KNeighborsClassifier()              # classifier: Nearest neighbors
# model = LogisticRegression()                # classifier: logistic regression
# model = LinearDiscriminantAnalysis()        # classifier: Linear Discriminant Analysis (LDA)
model = svm.LinearSVC()                     # classifier: Support Vector Machine (SVM)- linear support vector classifier (SVC)

# Normalize 
scaler.fit(trainX)
trainXnorm = scaler.transform(trainX)
testXnorm = scaler.transform(testX)

# Fit model (Logistic Reg.) to the Training data
model.fit(trainXnorm, trainY)
    
# Predict the classes (y) from test features (X)
predictedY = model.predict(testXnorm)


# print quality metrics
print('\nTesting Classification Report:\n\n', classification_report(testY, predictedY))

print('\nTesting Confusion Matrix:\n')



Training set dimensions
(98640, 5)
(98640,)

Test set dimensions
(65760, 5)
(65760,)

Testing Classification Report:

               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     46602
         1.0       0.12      0.00      0.01      1368
         2.0       0.65      0.97      0.78      7513
         3.0       0.25      0.00      0.00      2286
         4.0       0.80      0.79      0.79      3817
         5.0       1.00      1.00      1.00      4168
         6.0       0.00      0.00      0.00         6

    accuracy                           0.93     65760
   macro avg       0.55      0.54      0.51     65760
weighted avg       0.90      0.93      0.91     65760


Testing Confusion Matrix:



C:\Users\lukem\anaconda3\envs\landmarks\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Producing the spectrogram of the signal allowed us to obtain a power spectral density of the signal over time. Using these spectral densities the total density of each band was calculated by summing all frequencies within the given band. The bands are specified as follows:
delta 0.4-4 Hz
theta 4-8 Hz
alpha 8-13 Hz
beta 13-50 Hz
By calculating the band power for each second and running a Support Machine Vector classifer specifcially the linear support vector classifier an accuracy of 93% and a weighted accuracy of 91%.